<a href="https://colab.research.google.com/github/PosgradoMNA/actividades-del-projecto-equipo_3/blob/main/Proyecto_16_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Equipo 3: Julio Osvaldo Hernández Bucio - A01794366 y Juan Antonio Melendres Villa - A00369017. Dra. María de la Paz Rico Fernández. Proyecto Final Avance 1.

In [114]:
import pandas as pd
import numpy as np

In [115]:
df = pd.read_csv('https://raw.githubusercontent.com/PosgradoMNA/actividades-del-projecto-equipo_3/main/Datos_de_calidad_del_agua_de_sitios_de_monitoreo_de_aguas_superficiales_2020.csv', encoding='latin1')
df.sample(10)

,CLAVE,SITIO,ORGANISMO_DE_CUENCA,ESTADO,MUNICIPIO,CUENCA,CUERPO DE AGUA,TIPO,SUBTIPO,LONGITUD,...,CONTAMINANTES,CUMPLE_CON_DBO,CUMPLE_CON_DQO,CUMPLE_CON_SST,CUMPLE_CON_CF,CUMPLE_CON_E_COLI,CUMPLE_CON_ENTEROC,CUMPLE_CON_OD,CUMPLE_CON_TOX,GRUPO
3085,OCPNO4613,RIO PIAXTLA 4,PACIFICO NORTE,SINALOA,SAN IGNACIO,RIO PIAXTLA 2,RIO PIAXTLA,LOTICO (HUMEDAL),RIO,-106.41370,...,"DQO,SST,CF,E_COLI,",SI,NO,NO,NO,NO,ND,SI,SI,LOTICO
1748,OCBAL2862,VASO PRESA YOSOCUTA OBRA DE TOMA,BALSAS,OAXACA,SAN MARCOS ARTEAGA,RIO MIXTECO,RIO MIXTECO,LENTICO (HUMEDAL),PRESA,-97.81091,...,NaN,SI,SI,SI,SI,SI,ND,SI,SI,LENTICO
2573,OCLSP3810M1,RIO LERMA EN ZALAMEA,LERMA SANTIAGO PACIFICO,JALISCO,LA BARCA,RIO LERMA 7,RIO LERMA,LOTICO,RIO,-102.50697,...,"DBO,DQO,CF,E_COLI,",NO,NO,SI,NO,NO,ND,SI,SI,LOTICO
604,DLGUE1269,ARNOLD-SIROCO,PACIFICO SUR,GUERRERO,ACAPULCO DE JUAREZ,LAGUNA DE COYUCA,BAHIA DE ACAPULCO,COSTERO,BAHIA,-99.89916,...,NaN,ND,ND,SI,ND,ND,SI,SI,SI,COSTERO
2703,OCLSP3962M1,PRESA LA YESCA II,LERMA SANTIAGO PACIFICO,JALISCO,HOSTOTIPAQUILLO,RIO SANTIAGO 3,PRESA LA YESCA,LENTICO,PRESA,-104.07268,...,"OD%S,",SI,SI,SI,SI,SI,ND,NO,SI,LENTICO
2641,OCLSP3892M1,BAHIA MELAQUE 2,LERMA SANTIAGO PACIFICO,JALISCO,CIHUATLAN,RIO PURIFICACION,BAHIA MELAQUE,COSTERO,BAHIA,-104.69447,...,NaN,ND,ND,SI,ND,ND,SI,SI,SI,COSTERO
1355,DLQUI2260,LAGUNA BACALAR 2,PENINSULA DE YUCATAN,QUINTANA ROO,BACALAR,LAGUNA BACALAR,LAGUNA BACALAR,LENTICO,LAGO,-88.23729,...,"DQO,",SI,NO,SI,SI,SI,ND,SI,SI,LENTICO
4048,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
795,DLAGU12,PRESA EL JOCOQUI COLA,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,SAN JOSE DE GRACIA,RIO SAN PEDRO,PRESA EL JOCOQUI,LENTICO,PRESA,-102.37846,...,"OD%F,",SI,SI,SI,SI,SI,ND,NO,SI,LENTICO
4134,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Creamos un array de index

In [116]:
index = np.arange(1, 4142, 1)

Podemos eliminar la columna clave y remplazarla con el ID de cada fila

In [117]:
# Find the name of the column by index
n = df.columns[0]

# Drop that column
df.drop(n, axis = 1, inplace = True)

# Put whatever series you want in its place
df[n] = index

Podemos inferir el estado con el dato del municipio en el que se encuentra, eliminamos la columna estado

In [118]:
df.drop('ESTADO', axis = 1, inplace = False)

,SITIO,ORGANISMO_DE_CUENCA,MUNICIPIO,CUENCA,CUERPO DE AGUA,TIPO,SUBTIPO,LONGITUD,LATITUD,PERIODO,...,CUMPLE_CON_DBO,CUMPLE_CON_DQO,CUMPLE_CON_SST,CUMPLE_CON_CF,CUMPLE_CON_E_COLI,CUMPLE_CON_ENTEROC,CUMPLE_CON_OD,CUMPLE_CON_TOX,GRUPO,CLAVE
0,PRESA EL SAUCILLO 100M AGUAS ARRIBA DE LA CORTINA,LERMA SANTIAGO PACIFICO,RINCON DE ROMOS,RIO SAN PEDRO,PRESA EL SAUCILLO,LENTICO,PRESA,-102.33911,22.24730,2020.0,...,SI,NO,SI,NO,SI,ND,SI,SI,LENTICO,1
1,"LOS CABOS SEG 22, 2 ISA10B",PENINSULA DE BAJA CALIFORNIA,LOS CABOS,SAN JOSE DEL CABO,OCEANO PACIFICO,COSTERO,OCEANO-MAR,-109.84290,22.90473,2020.0,...,ND,ND,SI,ND,ND,SI,SI,SI,COSTERO,2
2,"LOS CABOS SEG 22, 1 ISA10B",PENINSULA DE BAJA CALIFORNIA,LOS CABOS,SAN LUCAS,OCEANO PACIFICO,COSTERO,OCEANO-MAR,-109.86442,22.89880,2020.0,...,ND,ND,SI,ND,ND,SI,SI,SI,COSTERO,3
3,LOS CABOS 3,PENINSULA DE BAJA CALIFORNIA,LOS CABOS,SAN LUCAS,BAHIA SAN LUCAS,COSTERO,BAHIA,-109.88604,22.89609,2020.0,...,ND,ND,SI,ND,ND,SI,SI,SI,COSTERO,4
4,LOS CABOS 1,PENINSULA DE BAJA CALIFORNIA,LOS CABOS,SAN LUCAS,BAHIA SAN LUCAS,COSTERO,BAHIA,-109.89657,22.87694,2020.0,...,ND,ND,SI,ND,ND,SI,SI,SI,COSTERO,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4137
4137,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4138
4138,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4139
4139,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4140


In [119]:
df.head(50)

,SITIO,ORGANISMO_DE_CUENCA,ESTADO,MUNICIPIO,CUENCA,CUERPO DE AGUA,TIPO,SUBTIPO,LONGITUD,LATITUD,...,CUMPLE_CON_DBO,CUMPLE_CON_DQO,CUMPLE_CON_SST,CUMPLE_CON_CF,CUMPLE_CON_E_COLI,CUMPLE_CON_ENTEROC,CUMPLE_CON_OD,CUMPLE_CON_TOX,GRUPO,CLAVE
0,PRESA EL SAUCILLO 100M AGUAS ARRIBA DE LA CORTINA,LERMA SANTIAGO PACIFICO,AGUASCALIENTES,RINCON DE ROMOS,RIO SAN PEDRO,PRESA EL SAUCILLO,LENTICO,PRESA,-102.33911,22.24730,...,SI,NO,SI,NO,SI,ND,SI,SI,LENTICO,1
1,"LOS CABOS SEG 22, 2 ISA10B",PENINSULA DE BAJA CALIFORNIA,BAJA CALIFORNIA SUR,LOS CABOS,SAN JOSE DEL CABO,OCEANO PACIFICO,COSTERO,OCEANO-MAR,-109.84290,22.90473,...,ND,ND,SI,ND,ND,SI,SI,SI,COSTERO,2
2,"LOS CABOS SEG 22, 1 ISA10B",PENINSULA DE BAJA CALIFORNIA,BAJA CALIFORNIA SUR,LOS CABOS,SAN LUCAS,OCEANO PACIFICO,COSTERO,OCEANO-MAR,-109.86442,22.89880,...,ND,ND,SI,ND,ND,SI,SI,SI,COSTERO,3
3,LOS CABOS 3,PENINSULA DE BAJA CALIFORNIA,BAJA CALIFORNIA SUR,LOS CABOS,SAN LUCAS,BAHIA SAN LUCAS,COSTERO,BAHIA,-109.88604,22.89609,...,ND,ND,SI,ND,ND,SI,SI,SI,COSTERO,4
4,LOS CABOS 1,PENINSULA DE BAJA CALIFORNIA,BAJA CALIFORNIA SUR,LOS CABOS,SAN LUCAS,BAHIA SAN LUCAS,COSTERO,BAHIA,-109.89657,22.87694,...,ND,ND,SI,ND,ND,SI,SI,SI,COSTERO,5
5,BAHIA CABO SAN LUCAS,PENINSULA DE BAJA CALIFORNIA,BAJA CALIFORNIA SUR,LOS CABOS,SAN LUCAS,BAHIA SAN LUCAS,COSTERO,BAHIA,-109.90250,22.88000,...,ND,ND,SI,ND,ND,SI,SI,SI,COSTERO,6
6,LOS CABOS 2,PENINSULA DE BAJA CALIFORNIA,BAJA CALIFORNIA SUR,LOS CABOS,SAN LUCAS,BAHIA SAN LUCAS,COSTERO,BAHIA,-109.90540,22.88310,...,ND,ND,SI,ND,ND,SI,SI,SI,COSTERO,7
7,LAGUNA TODOS SANTOS - CENTRO DE LA LAGUNA,PENINSULA DE BAJA CALIFORNIA,BAJA CALIFORNIA SUR,LA PAZ,TODOS SANTOS,TODOS SANTOS,COSTERO,LAGUNA,-110.23902,23.43930,...,ND,ND,SI,ND,ND,NO,NO,SI,COSTERO,8
8,MANANTIAL SAN REGIS,PENINSULA DE BAJA CALIFORNIA,BAJA CALIFORNIA SUR,LA PAZ,TODOS SANTOS,TODOS SANTOS,LOTICO,ARROYO,-110.22379,23.45805,...,SI,SI,SI,NO,SI,ND,SI,SI,LOTICO,9
9,AGUA CALIENTE II,PENINSULA DE BAJA CALIFORNIA,BAJA CALIFORNIA SUR,LOS CABOS,SANTIAGO,SANTIAGO,LOTICO,ARROYO,-109.80786,23.43995,...,SI,SI,SI,SI,SI,ND,NO,SI,LOTICO,10


In [120]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4141 entries, 0 to 4140
Data columns (total 55 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   SITIO                   3493 non-null   object 
 1   ORGANISMO_DE_CUENCA     3493 non-null   object 
 2   ESTADO                  3493 non-null   object 
 3   MUNICIPIO               3493 non-null   object 
 4   CUENCA                  3492 non-null   object 
 5   CUERPO DE AGUA          3479 non-null   object 
 6   TIPO                    3493 non-null   object 
 7   SUBTIPO                 3479 non-null   object 
 8   LONGITUD                3493 non-null   float64
 9   LATITUD                 3493 non-null   float64
 10  PERIODO                 3493 non-null   float64
 11  DBO_mg/L                2581 non-null   object 
 12  CALIDAD_DBO             2581 non-null   object 
 13  DQO_mg/L                2581 non-null   object 
 14  CALIDAD_DQO             2581 non-null   

Eliminemos las columnas que tienen todos los valores nulos con base en lo observado arriba

In [121]:
df = df.drop('TOX_D_48_FON_UT', axis = 1, inplace = False)
df = df.drop('CALIDAD_TOX_D_48_FON', axis = 1, inplace = False)
df = df.drop('TOX_FIS_FON_15_UT', axis = 1, inplace = False)
df = df.drop('CALIDAD_TOX_FIS_FON_15', axis = 1, inplace = False)

Se puede observar que las ultimas filas vienen vacias

In [122]:
df.tail(100)

,SITIO,ORGANISMO_DE_CUENCA,ESTADO,MUNICIPIO,CUENCA,CUERPO DE AGUA,TIPO,SUBTIPO,LONGITUD,LATITUD,...,CUMPLE_CON_DBO,CUMPLE_CON_DQO,CUMPLE_CON_SST,CUMPLE_CON_CF,CUMPLE_CON_E_COLI,CUMPLE_CON_ENTEROC,CUMPLE_CON_OD,CUMPLE_CON_TOX,GRUPO,CLAVE
4041,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4042
4042,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4043
4043,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4044
4044,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4045
4045,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4046
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4136,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4137
4137,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4138
4138,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4139
4139,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4140


Buscamos el indice del ultimo registro que tiene informacion

In [123]:
indexRow = df.index[df['SITIO'] == 'RIO BRAVO 8 (PASO DE LOS INDIOS)']
indexRow

Int64Index([3492], dtype='int64')

In [124]:
df.loc[3492:].isnull().sum()

SITIO                     648
ORGANISMO_DE_CUENCA       648
ESTADO                    648
MUNICIPIO                 648
CUENCA                    648
CUERPO DE AGUA            648
TIPO                      648
SUBTIPO                   648
LONGITUD                  648
LATITUD                   648
PERIODO                   648
DBO_mg/L                  648
CALIDAD_DBO               648
DQO_mg/L                  648
CALIDAD_DQO               648
SST_mg/L                  648
CALIDAD_SST               648
COLI_FEC_NMP_100mL        648
CALIDAD_COLI_FEC          648
E_COLI_NMP_100mL          648
CALIDAD_E_COLI            648
ENTEROC_NMP_100mL         649
CALIDAD_ENTEROC           649
OD_PORC                   648
CALIDAD_OD_PORC           648
OD_PORC_SUP               649
CALIDAD_OD_PORC_SUP       649
OD_PORC_MED               649
CALIDAD_OD_PORC_MED       649
OD_PORC_FON               649
CALIDAD_OD_PORC_FON       649
TOX_D_48_UT               648
CALIDAD_TOX_D_48          648
TOX_V_15_U

In [125]:
4141 - 649

3492

In [126]:
df1 = df.drop(df.index[3493:])

In [127]:
df.shape

(4141, 51)

In [128]:
df1.shape

(3493, 51)

In [129]:
df1.isnull().sum()

SITIO                        0
ORGANISMO_DE_CUENCA          0
ESTADO                       0
MUNICIPIO                    0
CUENCA                       1
CUERPO DE AGUA              14
TIPO                         0
SUBTIPO                     14
LONGITUD                     0
LATITUD                      0
PERIODO                      0
DBO_mg/L                   912
CALIDAD_DBO                912
DQO_mg/L                   912
CALIDAD_DQO                912
SST_mg/L                     4
CALIDAD_SST                  4
COLI_FEC_NMP_100mL         911
CALIDAD_COLI_FEC           911
E_COLI_NMP_100mL           911
CALIDAD_E_COLI             911
ENTEROC_NMP_100mL         2589
CALIDAD_ENTEROC           2589
OD_PORC                   1696
CALIDAD_OD_PORC           1696
OD_PORC_SUP               1874
CALIDAD_OD_PORC_SUP       1874
OD_PORC_MED               3006
CALIDAD_OD_PORC_MED       3006
OD_PORC_FON               2547
CALIDAD_OD_PORC_FON       2547
TOX_D_48_UT               1677
CALIDAD_

Faltan los valores < 1 y esas cosas

In [130]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']

In [131]:
numericColumns = list(df1.select_dtypes(include=numerics).columns.values)

In [132]:
df2 = df1[df1.isnull().any(axis=1)]
df2.loc[:,numericColumns]

,LONGITUD,LATITUD,PERIODO,CLAVE
0,-102.33911,22.24730,2020.0,1
1,-109.84290,22.90473,2020.0,2
2,-109.86442,22.89880,2020.0,3
3,-109.88604,22.89609,2020.0,4
4,-109.89657,22.87694,2020.0,5
...,...,...,...,...
3488,-99.42142,26.78971,2020.0,3489
3489,-99.53064,27.43714,2020.0,3490
3490,-99.50727,27.49901,2020.0,3491
3491,-99.52221,27.49631,2020.0,3492


In [133]:
df1['CUMPLE_CON_DBO'] = df1['CUMPLE_CON_DBO'].replace('SI',int(1))
df1['CUMPLE_CON_DBO'] = df1['CUMPLE_CON_DBO'].replace('NO',int(0))
df1['CUMPLE_CON_DBO'] = df1['CUMPLE_CON_DBO'].replace('ND',np.NaN)


0     1.0
1     NaN
2     NaN
3     NaN
4     NaN
     ... 
95    NaN
96    NaN
97    NaN
98    NaN
99    NaN
Name: CUMPLE_CON_DBO, Length: 100, dtype: float64

In [136]:
df1['CUMPLE_CON_DQO'] = df1['CUMPLE_CON_DQO'].replace('SI',int(1))
df1['CUMPLE_CON_DQO'] = df1['CUMPLE_CON_DQO'].replace('NO',int(0))
df1['CUMPLE_CON_DQO'] = df1['CUMPLE_CON_DQO'].replace('ND',np.NaN)

In [137]:
df1['CUMPLE_CON_SST'] = df1['CUMPLE_CON_SST'].replace('SI',int(1))
df1['CUMPLE_CON_SST'] = df1['CUMPLE_CON_SST'].replace('NO',int(0))
df1['CUMPLE_CON_SST'] = df1['CUMPLE_CON_SST'].replace('ND',np.NaN)

In [138]:
df1['CUMPLE_CON_CF'] = df1['CUMPLE_CON_CF'].replace('SI',int(1))
df1['CUMPLE_CON_CF'] = df1['CUMPLE_CON_CF'].replace('NO',int(0))
df1['CUMPLE_CON_CF'] = df1['CUMPLE_CON_CF'].replace('ND',np.NaN)

In [139]:
df1['CUMPLE_CON_E_COLI'] = df1['CUMPLE_CON_E_COLI'].replace('SI',int(1))
df1['CUMPLE_CON_E_COLI'] = df1['CUMPLE_CON_E_COLI'].replace('NO',int(0))
df1['CUMPLE_CON_E_COLI'] = df1['CUMPLE_CON_E_COLI'].replace('ND',np.NaN)

In [140]:
df1['CUMPLE_CON_ENTEROC'] = df1['CUMPLE_CON_ENTEROC'].replace('SI',int(1))
df1['CUMPLE_CON_ENTEROC'] = df1['CUMPLE_CON_ENTEROC'].replace('NO',int(0))
df1['CUMPLE_CON_ENTEROC'] = df1['CUMPLE_CON_ENTEROC'].replace('ND',np.NaN)

In [141]:
df1['CUMPLE_CON_OD'] = df1['CUMPLE_CON_OD'].replace('SI',int(1))
df1['CUMPLE_CON_OD'] = df1['CUMPLE_CON_OD'].replace('NO',int(0))
df1['CUMPLE_CON_OD'] = df1['CUMPLE_CON_OD'].replace('ND',np.NaN)

In [142]:
df1['CUMPLE_CON_TOX'] = df1['CUMPLE_CON_TOX'].replace('SI',int(1))
df1['CUMPLE_CON_TOX'] = df1['CUMPLE_CON_TOX'].replace('NO',int(0))
df1['CUMPLE_CON_TOX'] = df1['CUMPLE_CON_TOX'].replace('ND',np.NaN)